In [131]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import re 
import os 

In [132]:
filePath = os.path.join('data/DataAnalyst.csv')

In [133]:
df = pd.read_csv(filePath)

In [134]:
df.head(2)

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,Easy Apply
0,0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),Are you eager to roll up your sleeves and harn...,3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,1961,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),-1,True
1,1,Quality Data Analyst,$37K-$66K (Glassdoor est.),Overview\n\nProvides analytical and technical ...,3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,1893,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,-1


In [135]:
# removed spaces and lower cased all column names to make them uniform
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

In [136]:
# removed unnecessary column 
df = df.drop('unnamed:_0',axis=1)

In [137]:
# inspect data 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2253 entries, 0 to 2252
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   job_title          2253 non-null   object 
 1   salary_estimate    2253 non-null   object 
 2   job_description    2253 non-null   object 
 3   rating             2253 non-null   float64
 4   company_name       2252 non-null   object 
 5   location           2253 non-null   object 
 6   headquarters       2253 non-null   object 
 7   size               2253 non-null   object 
 8   founded            2253 non-null   int64  
 9   type_of_ownership  2253 non-null   object 
 10  industry           2253 non-null   object 
 11  sector             2253 non-null   object 
 12  revenue            2253 non-null   object 
 13  competitors        2253 non-null   object 
 14  easy_apply         2253 non-null   object 
dtypes: float64(1), int64(1), object(13)
memory usage: 264.1+ KB


In [138]:
df['lower'] = df['salary_estimate'].apply(lambda x: pd.Series(re.findall(r'\d+',str(x).split('-')[0]),dtype='float64'))
df['upper'] = df['salary_estimate'].apply(lambda x: pd.Series(re.findall(r'\d+',str(x).split('-')[1]),dtype='float64'))

In [139]:
df['city'] = df['location'].apply(lambda x: x.split(',')[0].strip())
df['state'] = df['location'].apply(lambda x: x.split(',')[1].strip())
df['state'] = df['state'].replace('Arapahoe','CO')

In [140]:
df['industry'].value_counts()

-1                                          353
IT Services                                 325
Staffing & Outsourcing                      323
Health Care Services & Hospitals            151
Computer Hardware & Software                111
                                           ... 
Sporting Goods Stores                         1
News Outlet                                   1
Consumer Electronics & Appliances Stores      1
Consumer Product Rental                       1
Catering & Food Service Contractors           1
Name: industry, Length: 89, dtype: int64

In [144]:
# obtain the average salary expecation
df['avg_salary'] = df['upper'].add(df['lower']).div(2)